In [1]:
import logging
import warnings
from pprint import pprint

import numpy as np
from openeye import oechem
from openff.qcsubmit.common_structures import QCSpec, PCMSettings
from openff.qcsubmit.factories import OptimizationDatasetFactory
from openff.qcsubmit.workflow_components import StandardConformerGenerator
from openforcefield.topology import Molecule
from tqdm import tqdm

In [2]:
# Warnings that tell us we have undefined stereo and charged molecules
logging.getLogger("openforcefield").setLevel(logging.ERROR)
warnings.simplefilter("ignore")

# Dataset Preperation

Load in the SMILES patterns of the molecules to include:

In [3]:
with open("molecules.smi") as file:
    smiles_patterns = file.read().split("\n")
    smiles_patterns = [pattern for pattern in smiles_patterns if len(pattern) > 0]

Load in the molecules to be optimized:

In [4]:
molecules = [
    Molecule.from_smiles(smiles_pattern)
    for smiles_pattern in tqdm(smiles_patterns)
]

100%|██████████| 200/200 [00:00<00:00, 647.82it/s] 


Prepare the main dataset from the molecule list.

In [5]:
dataset_factory = OptimizationDatasetFactory(
    qc_specifications={
        "hf-6-31G*": QCSpec(
            method="hf",
            basis="6-31G*",
            spec_name="hf-6-31G*",
            spec_description=(
                "The quantum chemistry specification used to generate the original AM1BCCs."
            ),
        ),
        "resp-2-vacuum": QCSpec(
            method="pw6b95",
            basis="aug-cc-pV(D+d)Z",
            spec_name="resp-2-vacuum",
            spec_description=(
                "The quantum chemistry specification used in the RESP2 publication "
                "for the vacuum (i.e. no PCM) calculations."
            ),
        ),
        "resp-2-water": QCSpec(
            method="pw6b95",
            basis="aug-cc-pV(D+d)Z",
            spec_name="resp-2-water",
            spec_description=(
                "The quantum chemistry specification used in the RESP2 publication "
                "for the aqueous (i.e. with PCM) calculations."
            ),
            implicit_solvent=PCMSettings(
                units="angstrom",
                cavity_Type="GePol",
                cavity_Area=0.3,
                cavity_Scaling=True,
                cavity_RadiiSet="Bondi",
                cavity_Mode="Implicit",
                medium_SolverType="CPCM",
                medium_Solvent="Water",
            )
        ),
    }
)

dataset_factory.add_workflow_component(
    StandardConformerGenerator(max_conformers=10, rms_cutoff=0.1, clear_existing=True)
)

dataset = dataset_factory.create_dataset(
    dataset_name="OpenFF BCC Refit Study COH v2.0",
    tagline="Optimizations of diverse, para-substituted aniline derivatives.",
    description="A data set curated for the initial stage of the on-going OpenFF "
    "study which aims to co-optimize the AM1BCC bond charge correction (BCC) "
    "parameters against an experimental training set of density and enthalpy of "
    "mixing data points and a QM training set of electric field data."
    "\n\n"
    "The initial data set is limited to only molecules composed of C, O, H. This "
    "limited scope significantly reduces the number of BCC parameters which must "
    "be retrained, thus allowing for easier convergence of the initial "
    "optimizations."
    "\n\n"
    "The included molecules we combinatorially generated to cover a range of "
    "alcohol, ether, and carbonyl containing molecules.",
    molecules=molecules,
)

dataset.metadata.submitter = "simonboothroyd"
dataset.metadata.long_description_url = (
    "https://github.com/openforcefield/qca-dataset-submission/tree/master/"
    "submissions/"
    "2021-06-22-OpenFF-BCC-Refit-Study-COH-v2.0"
)

dataset.provenance["constructure"] = "0.0.1"

Preparation                   : 100%|█████████| 200/200 [00:08<00:00, 24.36it/s]


Make sure the molecules in the dataset match the input molecules

In [7]:
old_smiles = {Molecule.from_smiles(smiles).to_smiles(isomeric=False) for smiles in smiles_patterns}
new_smiles = {molecule.to_smiles(isomeric=False) for molecule in dataset.molecules}

assert len(old_smiles.symmetric_difference(new_smiles)) == 0

Describe the molecule in the dataset

In [8]:
confs = np.array([len(mol.conformers) for mol in dataset.molecules])

print("Number of unique molecules       ", dataset.n_molecules)
print("Number of filtered molecules     ", dataset.n_filtered)
print("Number of conformers             ", dataset.n_records)
print("Number of conformers min mean max",
      confs.min(), "{:6.2f}".format(confs.mean()), confs.max())

masses = []
for molecule in dataset.molecules:
    oemol = molecule.to_openeye()
    mass = oechem.OECalculateMolecularWeight(oemol)
    masses.append(mass)

print(f'Mean molecular weight: {np.mean(np.array(masses)):.2f}')
print(f'Max molecular weight: {np.max(np.array(masses)):.2f}')
print("Charges:", sorted(set(m.total_charge/m.total_charge.unit for m in dataset.molecules)))

Number of unique molecules        200
Number of filtered molecules      0
Number of conformers              1550
Number of conformers min mean max 1   7.75 10
Mean molecular weight: 117.62
Max molecular weight: 204.31
Charges: [0.0]


Describe the dataset

In [9]:
pprint(dataset.metadata.dict())

{'collection_type': 'OptimizationDataset',
 'creation_date': datetime.date(2021, 6, 22),
 'dataset_name': 'OpenFF BCC Refit Study COH v2.0',
 'elements': {'O', 'H', 'C'},
 'long_description': 'A data set curated for the initial stage of the on-going '
                     'OpenFF study which aims to co-optimize the AM1BCC bond '
                     'charge correction (BCC) parameters against an '
                     'experimental training set of density and enthalpy of '
                     'mixing data points and a QM training set of electric '
                     'field data.\n'
                     '\n'
                     'The initial data set is limited to only molecules '
                     'composed of C, O, H. This limited scope significantly '
                     'reduces the number of BCC parameters which must be '
                     'retrained, thus allowing for easier convergence of the '
                     'initial optimizations.\n'
                     '\n'
  

In [10]:
for spec, obj in dataset.qc_specifications.items():
    print("Spec:", spec)
    pprint(obj.dict())

Spec: hf-6-31G*
{'basis': '6-31G*',
 'implicit_solvent': None,
 'method': 'hf',
 'program': 'psi4',
 'spec_description': 'The quantum chemistry specification used to generate the '
                     'original AM1BCCs.',
 'spec_name': 'hf-6-31G*',
 'store_wavefunction': 'none'}
Spec: resp-2-vacuum
{'basis': 'aug-cc-pV(D+d)Z',
 'implicit_solvent': None,
 'method': 'pw6b95',
 'program': 'psi4',
 'spec_description': 'The quantum chemistry specification used in the RESP2 '
                     'publication for the vacuum (i.e. no PCM) calculations.',
 'spec_name': 'resp-2-vacuum',
 'store_wavefunction': 'none'}
Spec: resp-2-water
{'basis': 'aug-cc-pV(D+d)Z',
 'implicit_solvent': {'cavity_Area': 0.3,
                      'cavity_MinRadius': 52.917721067,
                      'cavity_Mode': 'Implicit',
                      'cavity_RadiiSet': 'Bondi',
                      'cavity_Scaling': True,
                      'cavity_Type': 'GePol',
                      'codata': 2010,
        

In [11]:
pprint(dataset.scf_properties)

[<SCFProperties.Dipole: 'dipole'>,
 <SCFProperties.Quadrupole: 'quadrupole'>,
 <SCFProperties.WibergLowdinIndices: 'wiberg_lowdin_indices'>,
 <SCFProperties.MayerIndices: 'mayer_indices'>]


Export the dataset.

In [12]:
dataset.export_dataset("dataset.json.bz2")
dataset.molecules_to_file("dataset.smi", "smi")

dataset.visualize("dataset.pdf", columns=8)